In [1]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
from PIL import Image
from scipy import misc
from sklearn.model_selection import train_test_split
import keras
# from keras.applications import *
from keras import applications, optimizers
from keras.models import Sequential
from keras.models import Model 
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

C:\Users\burt\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
name = {}
for line in open("name.txt"):
    file, classfi = line.strip().split(' ')
    name[file] = classfi

In [3]:
img_size = 75
batch_size = 32

In [4]:
ori_imgs = []
ori_label = []
for fn in glob.glob('knots/*.ppm'):
    new_img = Image.open(fn)
    
    file = fn.split('\\')[1]
    
    new_img = misc.imresize(new_img, (img_size, img_size, 3)) # 改大小
    new_img = Image.fromarray(new_img, 'RGB') # numpy to IMG
#     plt.imshow(new_img)
#     plt.show()
    print(file, name[file], np.array(new_img).shape)
# 
    ori_imgs.append(new_img)
    ori_label.append(name[file])

C:\Users\burt\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


knot0001.ppm dry_knot (75, 75, 3)
knot0002.ppm dry_knot (75, 75, 3)
knot0003.ppm dry_knot (75, 75, 3)
knot0004.ppm dry_knot (75, 75, 3)
knot0005.ppm dry_knot (75, 75, 3)
knot0006.ppm dry_knot (75, 75, 3)
knot0007.ppm dry_knot (75, 75, 3)
knot0008.ppm dry_knot (75, 75, 3)
knot0009.ppm encased_knot (75, 75, 3)
knot0010.ppm dry_knot (75, 75, 3)
knot0011.ppm dry_knot (75, 75, 3)
knot0012.ppm dry_knot (75, 75, 3)
knot0013.ppm dry_knot (75, 75, 3)
knot0014.ppm dry_knot (75, 75, 3)
knot0015.ppm dry_knot (75, 75, 3)
knot0016.ppm dry_knot (75, 75, 3)
knot0017.ppm dry_knot (75, 75, 3)
knot0018.ppm dry_knot (75, 75, 3)
knot0019.ppm dry_knot (75, 75, 3)
knot0020.ppm dry_knot (75, 75, 3)
knot0021.ppm dry_knot (75, 75, 3)
knot0022.ppm dry_knot (75, 75, 3)
knot0023.ppm dry_knot (75, 75, 3)
knot0024.ppm dry_knot (75, 75, 3)
knot0025.ppm dry_knot (75, 75, 3)
knot0026.ppm dry_knot (75, 75, 3)
knot0027.ppm dry_knot (75, 75, 3)
knot0028.ppm dry_knot (75, 75, 3)
knot0029.ppm dry_knot (75, 75, 3)
knot0030.p

knot0326.ppm sound_knot (75, 75, 3)
knot0327.ppm sound_knot (75, 75, 3)
knot0328.ppm sound_knot (75, 75, 3)
knot0329.ppm sound_knot (75, 75, 3)
knot0330.ppm sound_knot (75, 75, 3)
knot0331.ppm sound_knot (75, 75, 3)
knot0332.ppm sound_knot (75, 75, 3)
knot0333.ppm sound_knot (75, 75, 3)
knot0334.ppm sound_knot (75, 75, 3)
knot0335.ppm sound_knot (75, 75, 3)
knot0336.ppm sound_knot (75, 75, 3)
knot0337.ppm sound_knot (75, 75, 3)
knot0338.ppm sound_knot (75, 75, 3)
knot0339.ppm sound_knot (75, 75, 3)
knot0340.ppm sound_knot (75, 75, 3)
knot0341.ppm sound_knot (75, 75, 3)
knot0342.ppm sound_knot (75, 75, 3)
knot0343.ppm sound_knot (75, 75, 3)
knot0344.ppm sound_knot (75, 75, 3)
knot0345.ppm sound_knot (75, 75, 3)
knot0346.ppm sound_knot (75, 75, 3)
knot0347.ppm sound_knot (75, 75, 3)
knot0348.ppm sound_knot (75, 75, 3)
knot0349.ppm sound_knot (75, 75, 3)
knot0350.ppm sound_knot (75, 75, 3)
knot0351.ppm sound_knot (75, 75, 3)
knot0352.ppm sound_knot (75, 75, 3)
knot0353.ppm sound_knot (75,

In [5]:
images_per_class = {"sound_knot"   : 0, 
                    "dry_knot"     : 1,
                    "encased_knot" : 2, 
                    "decayed_knot" : 3,
                    "edge_knot"    : 4, 
                    "leaf_knot"    : 5,
                    "horn_knot"    : 6}

In [6]:
imgs = np.array([np.array(im) for im in ori_imgs])
imgs = imgs.reshape(imgs.shape[0], img_size, img_size, 3) / 255
# lb = LabelBinarizer().fit(ori_label)
label = keras.utils.to_categorical([images_per_class[ele] for ele in ori_label], 7)
labels = list(images_per_class.keys())

In [7]:
x_train, x_valid, y_train, y_valid = train_test_split(imgs, label, test_size=0.1, random_state=42)

In [8]:
weights = os.path.join('', 'test.h5')

In [9]:
datagen = ImageDataGenerator(horizontal_flip=True,
                             vertical_flip=True,
                             rotation_range=10,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2)
                                      
learning_rate = 0.0001
# batch_size = 32  #大多32

callbacks = [ EarlyStopping(monitor='val_loss', patience=10, verbose=0), 
              ModelCheckpoint(weights, monitor='val_loss', save_best_only=True, verbose=0),
              ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)]

base_model = applications.Xception(input_shape=(img_size, img_size, 3), include_top=False,
                      weights='xception_weights_tf_dim_ordering_tf_kernels_notop.h5', pooling='avg') # Average pooling reduces output dimensions
x = base_model.output

x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.5)(x)


# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(512, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(256, activation='relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.5)(x)



predictions = Dense(7, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze layers not in classifier due to loading imagenet weights
# for layer in base_model.layers:
#     layer.trainable = False

# print(model.summary())

# Load any existing weights
# if os.path.isfile(weights):
#     model.load_weights(weights)
    
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])


In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 75, 3)    0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 37, 37, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 37, 37, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 37, 37, 32)   0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con